In [29]:
import os 
import pandas as pd
import clickhouse_connect

In [ ]:
#truy nhập vào click house và lấy dữ liệu từ bảng tương ứng với điều kiện cụ thể
client = clickhouse_connect.get_client(
    host=os.getenv("CLICKHOUSE_HOST"),
    port=int(os.getenv("CLICKHOUSE_PORT")),
    username=os.getenv("CLICKHOUSE_USER"),
    password=os.getenv("CLICKHOUSE_PASSWORD")
)

query = """
SELECT *
FROM cem_network_test.result_full_flow_2025_31_12
WHERE delay_init_up > 0 and delay_init_up < 1500
"""
df = client.query_df(query)

In [31]:
# cắt dữ liệu theo từng 30p 
df['event_time'] = pd.to_datetime(df['from_time'], unit='s')
df['time_window'] = df['event_time'].dt.floor('30T')

C:\Users\This PC\AppData\Local\Temp\ipykernel_5796\2609242498.py:3: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df['time_window'] = df['event_time'].dt.floor('30T')


In [32]:
#đếm số lượng lỗi xem flow này có lỗi hay k 
FAIL_CODE=[1]   
df['is_fail'] = df['cs_cause_code'].isin(FAIL_CODE)
df.head()

,flow_idx,ts_idx,client_mac,client_ip,server_ip,proto,client_port,server_port,tls_domain,dns_domain,delay_init_up,delay_init_down,from_time,to_time,traffic_up,traffic_down,cs_cause_code,event_time,time_window,is_fail
0,2569358788,1767095649,7c:7b:68:1a:f0:a9,14.229.34.160,172.240.45.75,6,7018,443,servt.vidcrunch.com,_,258,1,1767095649,1767095698,1,6,0,2025-12-30 11:54:09,2025-12-30 11:30:00,False
1,2569599565,1767095693,4c:12:e8:af:ec:89,222.252.189.7,20.190.144.161,6,37337,443,login.live.com,_,90,1,1767095693,1767095695,5,5,0,2025-12-30 11:54:53,2025-12-30 11:30:00,False
2,2608141586,1767095524,14:ca:56:04:51:ff,113.177.47.184,157.240.199.35,256,64071,443,www.facebook.com,www.facebook.com,41,11,1767095524,1767095557,16,34,-2,2025-12-30 11:52:04,2025-12-30 11:30:00,False
3,2569574284,1767095687,24:43:e2:61:20:a9,14.249.163.214,142.250.197.227,6,56232,443,connectivitycheck.gstatic.com,connectivitycheck.gstatic.com,60,7,1767095687,1767095692,1,4,0,2025-12-30 11:54:47,2025-12-30 11:30:00,False
4,2569651048,1767095702,d4:9a:a0:49:d0:94,100.70.115.209,142.250.71.234,6,59664,443,semanticlocation-pa.googleapis.com,semanticlocation-pa.googleapis.com,40,4,1767095702,1767095703,2,2,0,2025-12-30 11:55:02,2025-12-30 11:30:00,False


In [33]:
# tính toán dữ liệu dựa trên các nhóm server_ip và time_window
agg = (
    df
    .groupby(['time_window', 'server_ip'])
    .agg(
        total_flow=('flow_idx', 'count'),
        total_failures=('is_fail', 'sum'),
        distinct_clients=('client_ip', 'nunique'),
        p_20_delayup=('delay_init_up', lambda x: x.quantile(0.2)),
        avg_delayup=('delay_init_up', 'mean')
    )
).reset_index()

agg['failure_rate'] = agg['total_failures'] / agg['total_flow']


In [34]:
agg.head()

,time_window,server_ip,total_flow,total_failures,distinct_clients,p_20_delayup,avg_delayup,failure_rate
0,2025-12-29 16:30:00,1.0.0.1,3,0,2,27.6,30.0,0.0
1,2025-12-29 16:30:00,1.1.1.1,10,0,6,34.0,34.0,0.0
2,2025-12-29 16:30:00,1.192.193.159,1,0,1,141.0,141.0,0.0
3,2025-12-29 16:30:00,1.193.223.9,4,0,2,193.0,247.5,0.0
4,2025-12-29 16:30:00,1.194.172.139,2,0,1,616.0,616.0,0.0


In [35]:
#chuẩn hóa dữ liệu trước khi lưu vào clickhouse
agg = agg.astype({
    'total_flow': 'int',
    'total_failures': 'int',
    'distinct_clients': 'int',
    'p_20_delayup': 'float',
    'avg_delayup': 'float',
    'failure_rate': 'float'
})

In [36]:
agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138431 entries, 0 to 138430
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   time_window       138431 non-null  datetime64[ns]
 1   server_ip         138431 non-null  string        
 2   total_flow        138431 non-null  int64         
 3   total_failures    138431 non-null  int64         
 4   distinct_clients  138431 non-null  int64         
 5   p_20_delayup      138431 non-null  float64       
 6   avg_delayup       138431 non-null  float64       
 7   failure_rate      138431 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(3), string(1)
memory usage: 8.4 MB


In [37]:
#lưu dữ liệu vào database clickhouse
client.insert_df(
    table='cem_kpi.agg_server_30m_test',
    df=agg
)
